In [13]:
import re
import numpy as np
import pandas as pd
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')
import pymssql
import openai
from langchain_groq import ChatGroq
from MilvusRetriever import MilvusRetriever
from dotenv import load_dotenv
import yaml
from datetime import datetime, date
from functools import lru_cache
import random
import uuid

import os
import json
import dateutil
import getpass
from typing import Annotated, Optional, Literal
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

from langgraph.graph.message import AnyMessage, add_messages
from langgraph.graph import END, StateGraph, START
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_core.messages import ToolMessage, HumanMessage, AIMessage
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.errors import NodeInterrupt
from langchain_openai import ChatOpenAI

from IPython.display import Image, display


In [14]:
# Load environment variables
load_dotenv()
data_cache = {}

# Chat history cache
chat_history_cache = {}
with open("config.yaml", 'r') as file:
        config = yaml.safe_load(file)
        database_config = config.get('database', {})
        user = database_config.get('username')
        password = database_config.get('password')

In [15]:
def validar_telefono(Telefono):
    """Valida el telefono de la persona
    """
    try:
        cnxn = pymssql.connect(server='192.168.50.38\\DW_FZ', database='DW_FZ', user=user, password=password)
        query4 = f"""
        SELECT CD.id_persona AS CEDULA, CD.numero_id AS NUMERO_CREDITO, CD.id_fecha_nacimiento AS FECHA_NACIMIENTO, DC.Telefono AS TELEFONO, DC.Correo AS CORREO, DC.Nombre as NOMBRE
        FROM DW_FZ.dbo.CRM_Datos_Cliente as DC
        LEFT JOIN DW_FZ.dbo.Dim_Cred_Demografico AS CD
        ON CD.id_persona = DC.Cedula  Where TELEFONO = '{Telefono}';
        """
        INFO_CL = pd.read_sql_query(query4, cnxn)
        if not INFO_CL.empty:
            print(f"Tiene cuenta validada correctamente.")
            return INFO_CL
        else:
            print(f"Cuenta no encontrada en la base de datos.")
            return False
    except Exception as e:
        print(f"Error al validar teléfono: {e}")
        return False

In [16]:
primer_mensaje_cliente = 'Hola, necesito ayuda con mi crédito'
telefono = "321474742"
df_cl = validar_telefono(telefono)
df_cl


Cuenta no encontrada en la base de datos.


False

In [17]:
def initial_message(primer_mensaje_cliente, telefono):
    df_cl = validar_telefono(telefono)
    if df_cl is not False:
        nombre = df_cl['NOMBRE'].values[0]
        palabras = nombre.lower().title().split()
        primera_palabra = palabras[0]
        segunda_palabra_desde_el_final = palabras[-2]
        nombre_inicial = primera_palabra + " " + segunda_palabra_desde_el_final
        primer_mensaje = f"""Hola! Bienvenido a Finanzauto, {nombre_inicial}, para confirmar que eres tu, necesito el correo electrónico que tienes registrado con nosotros o tu fecha de nacimiento."""
    else:
        primer_mensaje = f"""Hola! Bienvenido a Finanzauto, ¿Como te puedo ayudar el día de hoy?"""
    return primer_mensaje

In [18]:
initial_message(primer_mensaje_cliente,telefono)

Cuenta no encontrada en la base de datos.


'Hola! Bienvenido a Finanzauto, ¿Como te puedo ayudar el día de hoy?'

In [19]:
class RequiredInformation(BaseModel):
    provided_date: Optional[int] = Field(None,description="La fecha que proporcionó el usuario, debe estar en formato %AAAAMMDD%")
    provided_email: Optional[str] = Field(None,description="El email que proporcionó el usuario")
    
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    required_information: RequiredInformation
    validated: bool = False
    telephone : int

llm = ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'], model_name="llama3-70b-8192")

In [20]:
def verifier_node(message) -> RequiredInformation:
    loguer_system = """
                    Eres un verificador que mira el último mensaje humano y verifica si en este mensaje se encuentra una fecha de nacimiento o un correo electrónico.
                    La fecha puede estar en cualquier formato o puede estar escrita, fijate muy bien.
                    Si se encuentra la fecha, entonces conviertela a el formato %AAAAMMDD%.
                    Pon el campo o campos encontrados en la siguiente estructura de datos:
                    
                    class RequiredInformation(BaseModel):
                        provided_date: Optional[int] = Field(None,description="La fecha que proporcionó el usuario, debe estar en formato %AAAAMMDD%")
                        provided_email: Optional[str] = Field(None,description="El email que proporcionó el usuario")
                    
                    mensaje {message}
                """

    loguer_prompt = ChatPromptTemplate.from_template(loguer_system)
    loguer_chain = loguer_prompt | llm.with_structured_output(RequiredInformation)
    response = loguer_chain.invoke(
        {
            "message": message
        }
    )
    return response

def waiting_for_love(info_usuario: RequiredInformation):
    if info_usuario.provided_date is not None or info_usuario.provided_email is not None:
        return 'verification'
    else:
        return 'pafuera pa la calle'

    

In [ ]:
def info_general(state: State):
    

In [21]:
siguiente_mensaje = ' '

In [22]:
info_usuario = verifier_node(siguiente_mensaje)
info_usuario

RequiredInformation(provided_date=None, provided_email=None)

In [ ]:
x


In [ ]:
workflow = StateGraph(State)
workflow.add_node("loguer", loguer_node)
workflow.add_node("add_tool_msg", verifier_node)
workflow.add_node("verificado", verificado)


workflow.add_conditional_edges("loguer", get_state)
workflow.add_conditional_edges(START, manage_messages)
workflow.add_edge("add_tool_msg", "loguer")
workflow.add_edge("verificado", END)
workflow.add_edge(START, "loguer")